In [1]:
%%html
<style>
.output_wrapper, .output { height:auto !important; max-height:10000px; } /* your desired max-height here */
.output_scroll {     box-shadow:none !important; webkit-box-shadow:none !important; }
.noborder, .noborder tr, .noborder th, .noborder td { border-collapse: collapse; border: 0px; border: none!important; }
</style>

<h1>TODO: Barchart, df_trends?? </h1>
<h1>TODO: Show #flavors etc in summary table </h1>
<h2> TODO: Put labels in details </h2>

In [2]:
import sys, os
sys.path.append('./modules')
from Monitoring_Tools import *
from OpenStack_Tools  import *

In [4]:
platforms = ['demo8', 'poc1', 'poc2', 'nfv5']
#platforms = ['demo8', 'poc1' ]
#platforms = ['demo8']

HTML_OPS={}
STATUSES={}
DISK_USAGE={}

thresholds=[70,90]
colours=['lightgreen','orange','red']
for platform in platforms:
    inventory, HTML_OP, PING_STATUS, PING_PORTS_STATUS, ENDPOINTS_STATUS, VMS_STATUS = platformStatus(platform)
    DISK_USAGE = archive_df(inventory, platform)
    highestpc, highestpc_label, DF_TABLE_HTML = diskPCTable(DISK_USAGE, thresholds, colours)
    HIGHEST_DISK_PC = diskPCCell(highestpc, 0.5, thresholds, colours)
    HIGHEST_DISK_PC_HOST = '<b>{}</b>'.format(highestpc_label)

    HTML_OPS[platform] = HTML_OP + '<h2>Disk Usage</h2>' + DF_TABLE_HTML
    STATUSES[platform] = [ HIGHEST_DISK_PC, HIGHEST_DISK_PC_HOST, PING_STATUS, PING_PORTS_STATUS, ENDPOINTS_STATUS, VMS_STATUS ]

2 flavors
8 images
0 servers
ping(demo8co0[10.3.162.143:22]) ... ping(demo8co1[10.3.162.141:22]) ... ping(demo8co10[10.3.162.146:22]) ... ping(demo8co11[10.3.162.140:22]) ... ping(demo8co12[10.3.162.139:22]) ... ping(demo8co13[10.3.162.134:22]) ... ping(demo8co14[10.3.162.144:22]) ... ping(demo8co15[10.3.162.145:22]) ... ping(demo8co2[10.3.162.178:22]) ... ping(demo8co3[10.3.162.142:22]) ... ping(demo8co4[10.3.162.149:22]) ... ping(demo8co5[10.3.162.137:22]) ... ping(demo8co6[10.3.162.138:22]) ... ping(demo8co7[10.3.162.153:22]) ... ping(demo8co8[10.3.162.154:22]) ... ping(demo8co9[10.3.162.136:22]) ... ping(hlmvm[10.3.162.30:22]) ... ping(kvm[10.3.162.10:22]) ... ping(wrcl0[10.3.162.131:22]) ... ping(wrcl1[10.3.162.132:22]) ... 

In [5]:
display( HTML( '<h1><a name="RESULTS_STATUS" /> Platforms Status Summary </h1>' ))

DISP_STATUSES={}
DISP_STATUSES['--Platform']=[ 'Disk usage', 'Worst case Disk host', 'Ping', 'Ports open', 'Endpoints reachable', 'VMs ACTIVE' ]
for platform in platforms:
    LINK='<a href="#RESULTS_' + platform + '">' + platform +'</a>'
    DISP_STATUSES[LINK]=STATUSES[platform]

display( HTML( DictTable._repr_html_(DISP_STATUSES, STATUS_HIGHLIGHTS) ))

display( HTML( '<h1>Platforms results </h1>' ))
for platform in platforms:
    display( HTML( '<a name="RESULTS_{}"></a>'.format(platform) ))
    HTML_OP = HTML_OPS[platform]
    display( HTML( HTML_OP )) 

In [ ]:
def showUptimes():
    for host in sorted(inventory['ssh_check']):    
        ip = inventory['hosts'][host]['ansible_host']
        user = inventory['hosts'][host]['ansible_user']
        pkey = inventory['hosts'][host]['ssh_key']

        stdout, stderr = ssh_command(host, ip, user, pkey, "uptime")

        #print("LINE=" + stdout)
        uptime = strip_uptime(stdout)
        print(host + ":" + uptime)

### TODO:
- Create status at top of page (without Javascript - so visible in email?)
- df processing to show trends (as html table, as pie chart)
- use of openstack clientmanager for equivalent to "openstack service list" status

In [ ]:
# TODO: 
# equivalent of openstack service list

# See: how service list is implemented:
#    https://github.com/openstack/python-openstackclient/blob/master/openstackclient/compute/v2/service.py
# (compute_client = client_manager.compute)
#
# See also: how to create/use a clientmanager:
#    https://github.com/openstack/python-openstackclient/blob/master/examples/osc-lib.py

#from openstackclient.common import clientmanager
#from openstackclient.common import utils